# Lab 3: Búsquedas facetadas

En este notebook vamos a explorar las diferentes capacidades de búsqueda que nos ofrece Elasticserach. En concreto vamos a ver como construir un buscador por facetas como el que vemos en la siguiente imagen.

<img src="../images/els/els31.jpg" alt="Busqueda autocompletada"/>

Para realizar los ejercicios propuestos en este notebook, puedes utilizar el Dev Tools de Kibana. Puedes acceder a él a través de este enlace: http://127.0.0.1:5601

Para poder construir este tipo de buscadores vamos a utilizar el framework de agregación de Elasticsearch y más en concreto las **Bucket Aggregations** que nos van a permitir organizar grandes datasets en gurpos dependiendo del valor de un campo.

# Búsquedas facetadas

Cuando implementamos un buscador con facetas, entendemos por faceta cada una de las propiedades que tienen nuestros resgistros por las que son susceptibles de ser filatradas en una búsqueda. En el ejemplo de la imagen, las facetas son cada una de las propiedades de nuestro producto.
Para implementar el buscador, lo que nos interesa es poder mostrar los diferentes valores de cada una de las propiedades, de tal forma que el usuario pueda seleccionar de forma sencilla los valores de cada propiedad por los que filtrar los resultados.

En el caso concreto de Elasticsearch, podemos ver las facetas como cada uno de los diferentes campos de nuestro documento. Por lo que a la hora de poponer las diferentes altenativas para los valares de cada una de las facetas, necesitamos poder realizar una consulta que nos devuelva los diferentes valores para los campos que representan nuestras facetas.

Para realizar esta consuta necesitamos poder realizar busquedas agregadas. Elasticsearch nos proporciona un framework de agregación con dos tipos principales de agregaciones:

* **Bucket aggregations**: nos van a permitir organizar grandes datasets en gurpos dependiendo del valor de un campo.
* **Metric aggregations**: nos permiten calacular métricas como count, sum, min, max, avg... sobre campos de tipo numérico.

Para construir los filtros facetados se utilizan as agregaciones de tipo Bucket Aggregation. 

## Bucket Aggregations

Las bucket aggregarions no generan métricas sobre los campos, sino que generan gurpos o buckets de documentos asociados a un critério que indica si un documento entra o no en ese bucket. Además nos indica cuantos documentos han entrado en cada uno de los buckets.

El número máximo de buckets que puede devolver una consulta está limitado a nivel de cluster por el parámetro de configuración search.max_buckets, por defecto 65.536.

Las agregaciones de tipo bucket generan como resultado buckets, donde cada bucket está asociado con una clave y a un criterio de selección. 

Cuando se ejecuta la agregación, todos los criterios asociados a cada bucket se evalúan sobre cada documento que encontramos en el contexto de búsqueda y si estos criterios se cumplen se incluye en el bucket. 

Al final de la ejecución se obtiene una lista de los buckets con el conjunto de documentos que pertenece a cada uno de ellos.

Dependiendo del tipo del campo sobre el que vamos ha hacer las facetas utilizaremos las siguientes agregaciones (en este notebook sólo vamos a ver las más comunes. Para ver el total de agregaciones que se pueden realizar en Elasticsearch consultar la siguiente página de la documentación: https://www.elastic.co/guide/en/elasticsearch/reference/8.4/search-aggregations-bucket.html):

* Texto:
    * **Terms Aggregation**: los buckets se generan de forma dinámica para cada uno de los valores únicos que se encuentran en los campos especificados en la agregación.
* Números:
    * **Range Aggregation**
    * **Histogram Aggregation**
* Fechas:
    * **Date Range Aggregation**
    * **Date Histogram Aggregation**


Como paso previo vamos a crear el índice y a insertar unos cuantos datos en él para poder realizar los ejercicios.

Vamos a utilizar la sentencia bulk como hicimos en el notebook anterior.

`POST /bookdb_index/book/_bulk`

# 0. Preparación de los datos

Antes de empezar a ver las distintas opciones que tenemos, vamos a crear el índice y a insertar unos cuantos datos en él para poder realizar los ejercicios.

`
PUT stock
{
  "mappings": {
    "properties": {
      "year": { "format": "year", "type": "date" },
      "album": { "type" : "text" },
      "artist": {
        "type": "text",
        "fields": {
          "keyword": { 
            "type":  "keyword"
          }
        }
      },
      "genre": { "type": "keyword" },
      "label": { "type" : "keyword" },
      "track": { "type": "integer" },
      "price": { "type": "float" }
    }
  }
}
`

`POST _bulk
{"index":{"_index":"stock"}}
{"year": 1989, "album": "Bleach", "artist": "Nirvana", "genre": "Grunge", "label": "Sub Pop", "track": 11, "price": 7.87}
{"index":{"_index":"stock"}}
{"year": 1991, "album": "Nevermind", "artist": "Nirvana", "genre": "Grunge", "label": "DGC", "tracks": 12, "price": 8.54}   
{"index":{"_index":"stock"}}
{"year": 1993 , "album": "In Utero", "artist": "Nirvana", "genre": "Grunge", "label": "DGC", "tracks": 12, "price": 10.99}
{"index":{"_index":"stock"}}
{"year": 1963, "album": "Please Please Me", "artist": "The Beatles", "label": "Parlophone", "genre": "Pop", "tracks": 12, "price": 13.28}
{"index":{"_index":"stock"}}
{"year": 1964, "album": "A Hard Day's Night", "artist": "The Beatles", "label": "Parlophone", "genre": "Pop", "tracks": 13, "price": 14.19}
{"index":{"_index":"stock"}}
{"year": 1965, "album": "Help!", "artist": "The Beatles", "label": "Parlophone", "genre": "Pop", "tracks": 12, "price": 11.45}
{"index":{"_index":"stock"}}
{"year": 1966, "album": "Revolver", "artist": "The Beatles", "label": "Parlophone", "genre": "Pop", "tracks": 12, "price": 7.65}
{"index":{"_index":"stock"}}
{"year": 1969, "album": "Abbey Road", "artist": "The Beatles", "label": "Apple Records", "genre": "Pop", "tracks": 16, "price": 21.99}
{"index":{"_index":"stock"}}
{"year": 1964, "album": "The Rolling Stones", "artist": "Ther Rolling Stones", "label": "Decca Records", "genre": "Rock", "tracks": 12, "price": 17.49}
{"index":{"_index":"stock"}}
{"year": 1971, "album": "Sticky Fingers", "artist": "Ther Rolling Stones", "label": "Rolling Stones Records", "genre": "Rock", "tracks": 10, "price": 15.99}
{"index":{"_index":"stock"}}
{"year": 1983, "album": "Undercover", "artist": "Ther Rolling Stones", "label": "Rolling Stones Records", "genre": "Rock", "tracks": 11, "price": 17.26}
{"index":{"_index":"stock"}}
{"year": 1997, "album": "Bridges to Babylon", "artist": "Ther Rolling Stones", "label": "Virgin Records", "genre": "Rock", "tracks": 12, "price": 9.49}
{"index":{"_index":"stock"}}
{"year": 1969, "album": "Space Oddity", "artist": "David Bowie", "label": "Philips, Mercury", "genre": "Rock", "tracks": 10, "price": 20.80}
{"index":{"_index":"stock"}}
{"year": 1970, "album": "The Man Who Sold the World", "artist": "David Bowie", "label": "Mercury", "genre": "Rock", "tracks": 10, "price": 15.55}
{"index":{"_index":"stock"}}
{"year": 1972, "album": "The Rise and Fall of Ziggy Stardust and the Spiders from Mars", "artist": "David Bowie", "label": "RCA", "genre": "Rock", "tracks": 12, "price": 21.76}
{"index":{"_index":"stock"}}
{"year": 1974, "album": "Diamond Dogs", "artist": "David Bowie", "label": "RCA", "genre": "Rock", "tracks": 11, "price": 9.34}
{"index":{"_index":"stock"}}
{"year": 1977, "album": "Heroes", "artist": "David Bowie", "label": "RCA", "genre": "Rock", "tracks": 12, "price": 11.55}
{"index":{"_index":"stock"}}
{"year": 1983, "album": "Let's Dance", "artist": "David Bowie", "label": "EMI", "genre": "Rock", "tracks": 8, "price": 12.45}
{"index":{"_index":"stock"}}
{"year": 2016, "album": "Blackstar", "artist": "David Bowie", "label": "Sony Records", "genre": "Rock", "tracks": 8, "price": 21.50}
{"index":{"_index":"stock"}}
{"year": 1988, "album": "Surfer Rosa", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 15, "price": 22.87}
{"index":{"_index":"stock"}}
{"year": 1989, "album": "Doolittle", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 15, "price": 10.90}
{"index":{"_index":"stock"}}
{"year": 1987, "album": "Come on Pilgrim", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 8, "price": 8.23}
{"index":{"_index":"stock"}}
{"year": 1989, "album": "Monkey Gone to Heaven", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 1, "price": 5.10}
{"index":{"_index":"stock"}}
{"year": 1989, "album": "Here Comes Your Man", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 1, "price": 6.50}
{"index":{"_index":"stock"}}
{"year": 1988, "album": "Gigantic", "artist": "Pixies", "label": "4AD", "genre": "Alternative", "tracks": 1, "price": 7.45}
{"index":{"_index":"stock"}}
{"year": 2010, "album": "Innerspeaker", "artist": "Tame Impala", "label": "Modular Recordings", "genre": "Rock", "tracks": 11, "price": 13.69}
{"index":{"_index":"stock"}}
{"year": 2012, "album": "Lonerism", "artist": "Tame Impala", "label": "Modular Recordings", "genre": "Alternative", "tracks": 12, "price": 11.99}
{"index":{"_index":"stock"}}
{"year": 2015, "album": "Currents", "artist": "Tame Impala", "label": "Modular Recordings", "genre": "Alternative", "tracks": 13, "price": 12.32}
{"index":{"_index":"stock"}}
{"year": 2020, "album": "The Slow Rush", "artist": "Tame Impala", "label": "Modular Recordings", "genre": "Dance", "tracks": 12, "price": 21.10}
{"index":{"_index":"stock"}}
{"year": 2004, "album": "The College Dropout", "artist": "Kanye West", "label": "Roc-A-Fella Records", "genre": "Hip Hop", "tracks": 21, "price": 10.99}
{"index":{"_index":"stock"}}
{"year": 2005, "album": "Late Registration", "artist": "Kanye West", "label": "Roc-A-Fella Records", "genre": "Hip Hop", "tracks": 21, "price": 23.42}
{"index":{"_index":"stock"}}
{"year": 2007, "album": "Graduation", "artist": "Kanye West", "label": "Roc-A-Fella Records", "genre": "Hip Hop", "tracks": 13, "price": 10.30}
{"index":{"_index":"stock"}}
{"year": 2008, "album": "808s & Heartbreak", "artist": "Kanye West", "label": "Roc-A-Fella Records", "genre": "Hip Hop", "tracks": 12, "price": 13.74}
`

## 1. Terms Aggregation

Utilizaremos terms aggregation cuando queremos extraer facetas de campos de tipo texto. En concreto, la agregación Terms Aggregation sólo funciona sobre campos de tipo **keyword**. Si el campo a nalalizar es de tipo text, podemos utilizar los fielddata.

Si lo que quermos es crear una búsqueda por la faceta género, realizaremos una agregación por el campo "genere". Ejecuta la siguiente sentencia en Kibana y vamos a analizar el resultado.

`GET stock/_search
{
    "aggs" : {
        "genres" : {
            "terms" : { "field" : "genre" } 
        }
    }
}
`

En la información devuelta vamos a fijarnos en el campo "aggregations". ¿Podrídas contestar a las siguietnes preguntas? 

1. Para la faceta "genres" ¿cuántos buckets diferentes hay?
2. Cuantos documentos ha encontrado que contenga el genero rock en la faceta género?
3. ¿Hay algún orden en el que devuelva los bickets?

Por defecto Elasticsearch devuelve los 10 buckets con más elementos. Si queremos cambiar este comportamiento, podemos utilizar el parámetro size:

`GET stock/_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "size" : 5
            }
        }
    }
}`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

Elasticsearch también nos permite cambiar el orden en el que devuelve los buckets. Por ejemplo, si queremos devolver los bockes ordenados de forma descendente por el número de documentos que contiene:

`GET stock/_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "order" : { "_count" : "asc" }
            }
        }
    }
}`

1. ¿Cómo ordena los buckts esta sentencia?

Pero además también podemos ordenar el resultado por el valor de la clave del bucket, por ejemplo, si queremos ordenar los resultados por orden alfabético:

`GET stock/_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "order" : { "_key" : "asc" }
            }
        }
    }
}`

También podemos especificar el número mínimo de documentos que tiene que haber en cada bucket para que se incluya en el resultado.

`GET stock/_search
{
    "aggs" : {
        "genre" : {
            "terms" : {
                "field" : "genre",
                "min_doc_count": 10
            }
        }
    }
}`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

### 1.1 Ejercicio: Realiza una búsqueda por dos facetas, género y discográfica.
### 1.2 Ejercicio: Excluye del resultado los buckets que no tengan más de 10 documentos.
### 1.3 Ejercicio: Ordena el resultado de los buckets por el orden alfabéico del nombre de la discográfica.

## 2. Range Aggregation

Se utilizan sobre campos numéricos y permiten definir el rango de valores que puede tener cada bucket. Para ello se especifica el valor máximo y mínimo entre los que tien que estar el valor de una faceta para que se incluya en ese bucket.

`
GET stock/_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "ranges" : [
                    { "to" : 10.0 },
                    { "from" : 10.0, "to" : 20.0 },
                    { "from" : 20.0 }
                ]
            }
        }
    }
}
`

1. ¿Cuantos buckets nos devuelve esta consulta?

Podemos decirle a Elasticsearch que incluya como nombre del bucket el rango definido utilizando el parámetro "keyed" en la query:

`
GET stock/_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "keyed" : true,
                "ranges" : [
                    { "to" : 10.0 },
                    { "from" : 10.0, "to" : 20.0 },
                    { "from" : 20.0 }
                ]
            }
        }
    }
}
`

1. ¿Qué nombre tienen ahora los buckets?

Si queremos darle la misma semántica a los nombres que en el dominio que estamos modelando, podemos definir un nombre a cada bucket:

`
GET stock/_search
{
    "aggs" : {
        "price_ranges" : {
            "range" : {
                "field" : "price",
                "keyed" : true,
                "ranges" : [
                    { "key" : "cheap", "to" : 10 },
                    { "key" : "average", "from" : 10, "to" : 20 },
                    { "key" : "expensive", "from" : 20 }
                ]
            }
        }
    }
}
`

### 2.1 Ejercicio: Realiza una búsqueda que devuleva el número de discos que hay función del número de tracks que tiene cada uno, SinglePlay de hasta 3, StandarPlay: de 3 a 10 y LongPlay: mas de 10. 
### 2.2 Ejercicio: Pon como nombre del buket SingelP, SandarP y LongP  según corresponda.

## 3. Date Range Aggregation

Funciona igual que la agregación Range que hemos visto antes, pero con campos de tipo fecha. Además permite expresiones específicas para fechas para definir los rangos.

Entra en este enlace para ver las expresiones matemáticas que se pueden utilizar: https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#date-math

`
POST /stock/_search?size=0
{
    "aggs": {
        "range": {
            "date_range": {
                "field": "year",
                "format": "year",
                "ranges": [
                    { "to": "1970" }, 
                    { "from": "1990" } 
                ]
            }
        }
    }
}
`

### 3.1 Ejercicio: Realiza una búsqueda que devuelva el número de discos que hay por década desde 1900 hasta la actualidad.
### 3.2 Ejercicio: Pon como nobre del bucket la década a la que pertenence el resultado.


## 4. Histogram Aggregation

Se tuiliza con campos de tipo numérico. Permite realizar buckets en función de la sitribución de los datos de un campo. Para ello es necesario definir el número fijo de valores que puede haber entre el valor máximo y mínimo del bucket.

Por ejemplo si tenemos en cuenta el campo precio de nuestro índice, si le indicamos que cree los buckets con un intervalo de 5 (5€), cuando evalúe el valor del campo precio para cada documento lo insertará en el bucket mas próximo por redondeo. Por ejemplo, para le valor 30 lo insertará en el bucket 30.

` 
POST /stock/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 5
            }
        }
    }
}
`

1. ¿Cómo se incrementan los valores de los buckets según el resultado?

Como pasara con el Terms Aggregation, se puede definir el número mínimo de documentos que tiene que tener el bucket para que aparezca en el resultado.

`
POST /stock/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 5,
                "min_doc_count" : 5
            }
        }
    }
}
`

También permiten que el nombre del bucket sea el rango del intervalo.

`
POST /stock/_search?size=0
{
    "aggs" : {
        "prices" : {
            "histogram" : {
                "field" : "price",
                "interval" : 5,
                "keyed" : true
            }
        }
    }
}
`

### 4.1 Ejercicio: Realiza una búsqueda que devuelva el número de discos que hay por rating con un intervalo de 10.

## 5. Date Histogram Aggregation

Es igual que el Histogram Aggregation, pero con campos de tipo fecha. En este caso los inervalos pueden ser:

* minute (m, 1m)
* hour (h, 1h)
* day (d, 1d)
* week (w, 1w)
* month (M, 1M)
* quarter (q, 1q)
* year (y, 1y)

`
POST /stock/_search?size=0
{
    "aggs" : {
        "sales_over_time" : {
            "date_histogram" : {
                "field" : "year",
                "calendar_interval" : "1y"
            }
        }
    }
}
`

### 5.1 Ejercicio: Realiza el ejercicio 3.1 pero con Date Histogram 

# Construir el buscador

Para poder terminar de implementar nuestro buscador, además de poder mostrar los valores de las diferentes facetas, no interesa poder filtrar los resultados por los valores de las facetas seleccionados y a su vez poder regenerar de forma dinámica los filtros de las facetas teniendo en cuenta el nuevo resultado tras aplicar el filtro. 

El lenguaje de consulta de Elasticsearch permite añadir un contexto de agregación junto al contexto de búsqueda y filtrado. Con el resultado de la búsqueda se adjunta el resultado de la agregación.

La agregación siempre se aplica al subconjunto de documentos que cumplen los criterios de búsqueda definidos en el contexto de búsqueda y filtrado.

## Construir el filtro de facetas

El primer paso para constuir el buscador es consutruir la consulta que nos devuelva las facetas y sus valores. 
Vamos a poner en práctica lo que hemos aprendido sobre las bucket aggregations.

Queremos construir un filtro de facetas con las siguientes facetas:
* Artista
* Género
* Precio por rango (cheap, average, expensive)

`
GET stock/_search
{
    "aggs" : {
        "genres" : {
            "terms" : {
                "field" : "genre",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "artist" : {
            "terms" : {
                "field" : "artist.keyword",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "price" : {
            "range" : {
                "field" : "price",
                "keyed" : true,
                "ranges" : [
                    { "key" : "cheap", "to" : 10 },
                    { "key" : "average", "from" : 10, "to" : 20 },
                    { "key" : "expensive", "from" : 20 }
                ]
            }
        }
    }
}
`

Una vez que construimos el filtro de facetas, lo que esperamos es que el usuario lo utilice, por tanto puede seleccionar algún valor del filtro. Por lo tanto como resultado de la búsqueda tendremos que devolver los documentos filtrados por el valor o valores seleccionados, junto con las nuevos valores para las facetas.

Vamos a ver como sería la consulta. Imaginemos que el usuario ha seleccionado el valor 'Rock' parala faceta Genre:

`
GET stock/_search
{
  "query": {
    "term": {
      "genre": {
        "value": "Rock",
        "case_insensitive": true
      }
    }    
  }, 
  "aggs" : {
      "genres" : {
          "terms" : {
              "field" : "genre",
              "size" : 10,
            "order" : { "_key" : "asc" }
          }
      },
      "artist" : {
          "terms" : {
              "field" : "artist.keyword",
              "size" : 10,
              "order" : { "_key" : "asc" }
          }
      },
      "price" : {
          "range" : {
              "field" : "price",
              "keyed" : true,
              "ranges" : [
                  { "key" : "cheap", "to" : 10 },
                  { "key" : "average", "from" : 10, "to" : 20 },
                  { "key" : "expensive", "from" : 20 }
              ]
          }
      }
  }
}
`